In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt

from os.path import join
from tqdm import tqdm
from keras.preprocessing import image
from sklearn.preprocessing import label_binarize
from sklearn.model_selection import train_test_split
from mpl_toolkits.axes_grid1 import ImageGrid

from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import SGD, Adam

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Read the labels.csv file

DATA_DIR = "../input/dog-breed-identification"

labels_all = pd.read_csv(join(DATA_DIR, "labels.csv"))
print(labels_all.shape)
labels_all.head()

In [ ]:
# Visualize the number of each breeds
breeds_all = labels_all["breed"]
plt.figure(figsize=(15,7))
g = sns.countplot(breeds_all, palette="icefire")
plt.title("Number of breed classes")
breed_counts = breeds_all.value_counts()
breed_counts

In [ ]:
# Selecting breeds

CLASS_NAMES = ['scottish_deerhound',
               'maltese_dog',
               'bernese_mountain_dog']

labels = labels_all[(labels_all['breed'].isin(CLASS_NAMES))]

labels = labels.reset_index()
labels.head()

In [ ]:
def read_img(img_id, train_or_test, size):
    """Read and resize image.
    # Arguments
        img_id: string, name of the file
        train_or_test: string 'train' or 'test'
        size: resize the original image
    # Returns
        Image as numpy array
    """
    img = image.load_img(join(DATA_DIR, train_or_test, '%s.jpg' % img_id), target_size=size)
    img = image.img_to_array(img)
    return img

In [ ]:
# Loading the training set

IMG_SIZE = 224

X_all = np.zeros((len(labels), IMG_SIZE, IMG_SIZE, 3), dtype='float32')
Y_all = label_binarize(labels['breed'], classes = CLASS_NAMES)

for i in tqdm(range(len(labels))):
    img = read_img(labels['id'][i], 'train', (IMG_SIZE, IMG_SIZE))
    x = np.expand_dims(img.copy(), axis=0)
    X_all[i] = x / 255.0
    
print('Train Images shape: {} size: {:,}'.format(X_all.shape, X_all.size))
print('One-hot encoded output shape: {} size: {:,}'.format(Y_all.shape, Y_all.size))

In [ ]:
# Show example images

IMG_GRID_ROWS = 4
IMG_GRID_COLUMNS = 4
num_of_images = IMG_GRID_ROWS * IMG_GRID_COLUMNS
fig = plt.figure(1, figsize=(IMG_GRID_COLUMNS * 4, IMG_GRID_ROWS * 4))
grid = ImageGrid(fig, 111, nrows_ncols=(IMG_GRID_ROWS, IMG_GRID_COLUMNS), axes_pad=0.05)

for i in range(num_of_images):
    ax = grid[i]
    ax.imshow(X_all[i,:,:,:])
    class_index = np.argmax(Y_all[i])
    ax.text(10, 200, ('LABEL: %s' % CLASS_NAMES[class_index]), backgroundcolor='w')
    ax.axis('off')
plt.show()

In [ ]:
# Building the Model

model = Sequential()

# 1. Convolution Layer + MaxPool
model.add(Conv2D(filters = 8, kernel_size = (5,5),
                 activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.2))

# 2. Convolution Layer + MaxPool
model.add(Conv2D(filters = 16, kernel_size = (3,3), 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.2))

# 3. Convolution Layer + MaxPool
model.add(Conv2D(filters = 32, kernel_size = (7,7),
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.2))

# 4. Convolution Layer + MaxPool
model.add(Conv2D(filters = 64, kernel_size = (5,5),
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.2))

# fully connected
model.add(Flatten())
model.add(Dense(2048, activation = "relu"))
model.add(Dropout(0.2))
model.add(Dense(2048, activation = "relu"))
model.add(Dropout(0.2))
model.add(Dense(2048, activation = "relu"))
model.add(Dropout(0.2))
model.add(Dense(1024, activation = "relu"))
model.add(Dropout(0.2))
model.add(Dense(len(CLASS_NAMES), activation = "softmax"))

learning_rate = 0.0005

model.compile(loss = categorical_crossentropy,
              optimizer = Adam(learning_rate),
              metrics=['accuracy'])

model.summary()

In [ ]:
# Splitting the training data set into training and validation data sets

X_train_and_val, X_test, Y_train_and_val, Y_test = train_test_split(X_all, Y_all, test_size = 0.1)

X_train, X_val, Y_train, Y_val = train_test_split(X_train_and_val, Y_train_and_val, test_size = 0.2)

In [ ]:
#Save the model during training 

save_at = "/kaggle/working/model.hdf5"
save_best = ModelCheckpoint (save_at, monitor='val_accuracy', verbose=0, save_best_only=True, save_weights_only=False, mode='max')

In [ ]:
# Data augmentation

datagen = ImageDataGenerator(
        rotation_range = 20,  # randomly rotate images
        zoom_range = 0.2, # Randomly zoom images
        width_shift_range = 0.25,  # randomly shift images horizontally
        height_shift_range = 0.25,  # randomly shift images vertically
        horizontal_flip = True,  # randomly flip images horizontally
        vertical_flip = False)  # randomly flip images vertically

datagen.fit(X_train)

In [ ]:
# Training the model

epochs = 300
batch_size = 20

# Normal training

#history = model.fit(X_train, Y_train,
#                    batch_size = batch_size,
#                    epochs = epochs,
#                    validation_data = (X_val, Y_val),
#                    callbacks = [save_best],)

# Training with data augmentation

history = model.fit_generator(datagen.flow(X_train,Y_train, batch_size=batch_size),
                              epochs = epochs,
                              validation_data = (X_val, Y_val),
                              callbacks = [save_best],
                              steps_per_epoch=X_train.shape[0] // batch_size)

In [ ]:
# Plot the training history

plt.figure(figsize=(12, 5))

plt.subplot(121)
plt.plot(history.history['loss'], color='r')
plt.plot(history.history['val_loss'], color='b')
plt.title('Model Loss', weight='bold', fontsize=16)
plt.ylabel('loss', weight='bold', fontsize=14)
plt.xlabel('epoch', weight='bold', fontsize=14)
plt.ylim(0.0, 1.2)
plt.xticks(weight='bold', fontsize=12)
plt.yticks(weight='bold', fontsize=12)
plt.legend(['train', 'val'], loc='upper right', prop={'size': 14})
plt.grid(color = 'y', linewidth='0.5')

plt.subplot(122)
plt.plot(history.history['accuracy'], color='r')
plt.plot(history.history['val_accuracy'], color='b')
plt.title('Model Accuracy', weight='bold', fontsize=16)
plt.ylabel('accuracy', weight='bold', fontsize=14)
plt.xlabel('epoch', weight='bold', fontsize=14)
plt.ylim(0.2, 1.0)
plt.xticks(weight='bold', fontsize=12)
plt.yticks(weight='bold', fontsize=12)
plt.legend(['train', 'val'], loc='lower right', prop={'size': 14})
plt.grid(color = 'y', linewidth='0.5')

plt.show()

In [ ]:
# Making predictions on the test dataset

model = load_model('/kaggle/working/model.hdf5')

Y_pred = model.predict(X_test)
score = model.evaluate(X_test, Y_test)
print('Accuracy over the test set: \n ', round((score[1]*100), 2), '%')

In [ ]:
# Show example predictions

IMG_GRID_ROWS = 6
IMG_GRID_COLUMNS = 4
num_of_images = IMG_GRID_ROWS * IMG_GRID_COLUMNS
fig = plt.figure(1, figsize=(IMG_GRID_COLUMNS * 4, IMG_GRID_ROWS * 4))
grid = ImageGrid(fig, 111, nrows_ncols=(IMG_GRID_ROWS, IMG_GRID_COLUMNS), axes_pad=0.05)

for i in range(num_of_images):
    ax = grid[i]
    ax.imshow(X_test[i,:,:,:])
    
    class_index_pred = np.argmax(Y_pred[i])
    class_index_val = np.argmax(Y_test[i])
    
    if class_index_val == class_index_pred:
        background_color = 'g'
    else:
        background_color = 'r'
        
    ax.text(10, 200, ('LABEL: %s' % (CLASS_NAMES[class_index_pred])),
            color = 'w',
            backgroundcolor = background_color)
    
    ax.axis('off')
plt.show()